In [88]:
def infoGain(P,N):
    import math
    return -P/(P+N)*math.log2(P/(P+N))-N/(P+N)*math.log2(N/(P+N))

In [89]:
def getpath(nested_dict, value, prepath=0):
    for k, v in nested_dict.items():
        path = prepath
        if v == value: 
            return path+1
        elif hasattr(v, 'items'): 
            p = getpath(v, value, path) 
            if p is not None:
                return p

In [90]:
def finditem(obj, key):
    if key in obj: return key
    for k, v in obj.items():
        if isinstance(v,dict):
            item = _finditem(v, key)
            if item is not None:
                return item

In [110]:
def insertNode(tree,Node,AttPosNeg,d):
    for k, v in tree.items():
        if k == d:
            NV = {}
            for i in AttPosNeg:
                for j in AttPosNeg[i]:
                    NV[j]='None'
                    if AttPosNeg[i][j] == 0:
                        NV[j]='No'
                        if AttPosNeg[i][j] != 0:
                            break
                    if AttPosNeg[i][j] == 0:
                        NV[j]='Yes'
                        break
            tree[k]={Node:NV}
        elif hasattr(v, 'items'):
            p = insertNode(v, Node,AttPosNeg,d)
            if p is 'None':
                return p
    return tree

In [101]:
def countConcept(data,attr,val):
    from collections import Counter
    cnt=Counter(x for x in data[attr])
    return cnt[val]

In [102]:
def countAttr(data,attr,val,concept,cVal):
    count={}
    C = data[concept]
    A = data[attr]
    for a in data.index:
        for v in val:
            if v not in count:
                count[v] = 0
            if(A[a]==v and C[a]==cVal):
                if v not in count:
                    count[v]=1
                else:
                    count[v]=count[v]+1
    return count

In [103]:
def getNextData(data,Node,d):
    iData = (data[Node] == d)
    nD = data[iData]
    nD = nD.drop(Node,axis=1)
    return nD

In [107]:
def getTree(data,AttributeList,concept,tree,d):
    flag = True
    Attr = {}
    Total=data.shape[0]
    for a in AttributeList:
        Attr[a] = list(set(data[a]))
 
    conceptVals = list(set(data[concept]))
    AttrCount = {}
    countC={}
    
    pathToNone = getpath(tree,'None')

    if(pathToNone>0):
        for val in Attr:
            AttrCount[val] = {}
            for cVal in conceptVals:
                countC[cVal] = 0
                countC[cVal] = countConcept(data,concept,cVal)
            for cVal in conceptVals:
                AttrCount[val][cVal] = countAttr(data,val,Attr[val],concept,cVal)
#         print(AttrCount)
        print(countC)
        EntropyAttr = {}
        AttPosNeg = {}
        ClassEntropy = infoGain(countC['Yes'],countC['No'])
#         print("Class Entropy : ",ClassEntropy)
        for att in AttrCount:
            InfoGain = {}
            ap = {}
            an = {}
            for c in conceptVals:
                for val in AttrCount[att][c]:
                    if c == 'No':
                        an[val] = AttrCount[att][c][val]
                    if c == 'Yes':
                        ap[val] = AttrCount[att][c][val]
            AttPosNeg[att]={}
            TotalInfo = {}
            AttPosNeg[att] = {'Yes':ap,'No':an}
            for val in AttrCount[att][c]:
                TotalInfo[val] = ap[val]+an[val]
                if ap[val]==0 or an[val]==0:
                    InfoGain[val]=0
                else:
                    InfoGain[val] = infoGain(ap[val],an[val])
#             print("Information Gain : ",InfoGain)
            for val in InfoGain:
                if att not in EntropyAttr:
                    EntropyAttr[att] = ( TotalInfo[val] / Total ) * InfoGain[val]
                else:
                    EntropyAttr[att] = EntropyAttr[att] + ( TotalInfo[val] / Total ) * InfoGain[val]
#         print("Attribute Entropy : ",EntropyAttr)
        Gain = {}
        for g in EntropyAttr:
            if g not in Gain:
                Gain[g] = ClassEntropy - EntropyAttr[g]
#         print("Gain : ",Gain)
        Node = max(Gain,key=Gain.get)
        print("Next root : ",Node)
        tree = insertNode(tree,Node,AttPosNeg[Node],d)
        print(tree)
        return Node,tree
    else:
        return None,tree

In [114]:
def main():
    import pandas as pd
    from pandas import DataFrame 
    from collections import Counter
    data = DataFrame.from_csv('play.csv')
    AttributeList=list(data)[:-1]
    num_of_attributes=len(AttributeList)
    concept=str(list(data)[-1])
    Node,tree = getTree(data,AttributeList,concept,{'root':'None'},'root')
    nDL = list(set(findItemsList(tree,'None')))
#     print(nDL)
    for d in nDL:
        newData = getNextData(data,Node,d)
        print(newData)
        AttributeList=list(newData)[:-1]
        Node,tree = getTree(newData,AttributeList,concept,tree,d)

In [115]:
main()

{'Yes': 9, 'No': 5}
Next root :  Outlook
{'root': {'Outlook': {'Overcast': 'Yes', 'Rain': 'None', 'Sunny': 'None'}}}
     Temperature Humidity    Wind PlayTennis
slno                                        
3           Mild     High    Weak        Yes
4           Cool   Normal    Weak        Yes
5           Cool   Normal  Strong         No
9           Mild   Normal    Weak        Yes
13          Mild     High  Strong         No
{'Yes': 3, 'No': 2}
Next root :  Wind
{'root': {'Outlook': {'Overcast': 'Yes', 'Rain': {'Wind': {'Strong': 'None', 'Weak': 'Yes'}}, 'Sunny': 'None'}}}
Empty DataFrame
Columns: [Outlook, Temperature, Humidity, PlayTennis]
Index: []
{}


KeyError: 'Yes'